In [2]:
import warnings
warnings.simplefilter("ignore")

Escalamiento de los datos

In [3]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt


<div class="alert alert-success">
  Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
</div>

Existen distintas estrategias de escalamiento de tus features, pero **la más común es la estandarización** donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [5]:
from sklearn.model_selection import train_test_split

X = pd.read_csv('vol/intermediate_results/X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
scaler.mean_

array([2.03867251e+03, 3.39681853e+07, 2.00214718e+03, 2.12867610e+00,
       1.08302857e+02, 1.03370260e+04, 4.18565685e+07, 6.45445094e+00])

In [10]:
scaler.scale_

array([1.19087725e+03, 4.19863962e+07, 1.21342357e+01, 8.02254689e-01,
       2.27568559e+01, 1.92117857e+04, 2.31546310e+08, 1.06994714e+00])

In [11]:
X.values

array([[0.00000000e+00, 4.25000000e+08, 2.00900000e+03, ...,
        4.83400000e+03, 2.37000000e+08, 7.90000000e+00],
       [1.00000000e+00, 3.06000000e+08, 2.00213073e+03, ...,
        1.43000000e+02, 4.04553863e+07, 7.10000000e+00],
       [2.00000000e+00, 3.00000000e+08, 2.00700000e+03, ...,
        4.83500000e+04, 3.00000000e+08, 7.10000000e+00],
       ...,
       [4.10100000e+03, 7.00000000e+03, 2.00500000e+03, ...,
        9.30000000e+01, 3.25000000e+03, 7.80000000e+00],
       [4.10200000e+03, 3.96700000e+03, 2.01200000e+03, ...,
        2.38600000e+03, 4.04553863e+07, 6.30000000e+00],
       [4.10300000e+03, 1.10000000e+03, 2.00400000e+03, ...,
        1.63000000e+02, 1.10000000e+03, 6.60000000e+00]])

In [12]:
scaler.transform(X_train)

array([[ 1.47649767, -0.78521112, -1.74276949, ..., -0.30705246,
        -0.17645096,  0.13603388],
       [-1.32731775,  0.9772645 ,  0.31751685, ..., -0.44358323,
         0.16473349,  0.41642156],
       [-0.43805734, -0.09451121, -0.34177478, ..., -0.37815464,
        -0.05120603, -0.33127893],
       ...,
       [ 1.48069626, -0.78521112,  0.2351054 , ..., -0.37190848,
        -0.17645096,  0.88373437],
       [-0.71768313,  0.14366117, -0.09454042, ..., -0.47824945,
        -0.00801813, -0.61166661],
       [ 0.88365739, -0.64707114,  0.2351054 , ..., -0.42775961,
        -0.15140197,  0.50988412]])

In [13]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [15]:
from sklearn.linear_model import Lasso
model = Lasso()
model_scaled = Lasso()

model.fit(X_train,y_train)
model_scaled.fit(X_train_scaled,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [17]:
print(model.score(X_test,y_test))
print(model_scaled.score(X_test_scaled,y_test))

0.564360355114472
0.5643603567551818


<div class="alert alert-success">
  Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
</div>

## Simplificar las transformaciones con pipelines

<div class="alert alert-info">
  Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación te recomendamos utilizar <code> sklearn.pipeline.make_pipeline </code> que permite encadenar transformaciones a tus modelos.
</div>

In [18]:
from sklearn.pipeline import make_pipeline
model_scaled = make_pipeline(StandardScaler(),Lasso())
model_scaled.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [19]:
print(model_scaled.score(X_test,y_test))

0.5643603567551818


## Crear nuevas features de forma automatica

In [22]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [23]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

``PolynomialFeatures`` transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [24]:
X.shape

(4104, 8)

In [25]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(4104, 45)

In [26]:
model_poly = make_pipeline(PolynomialFeatures(),
                          Lasso())
model_poly.fit(X_train,y_train)
model_poly.score(X_test,y_test)

0.603186592379081

In [27]:
model = Lasso()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.564360355114472

## Crear features categóricas

<div class="alert alert-success">
  En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.
</div>

Un mapeo del tipo $\{Perú, Chile, Colombia, Venezuela\} \rightarrow \{1, 2, 3, 4\}$ tiene el problema de asignarle un ordén a los valores posibles de la categoría. Este orden impacta de distintas maneras los algoritmos de Machine Learning, por ejemplo aquellos que dependen de la topología de $R^n$ y de la función de distancia entre puntos en este espacio, considerarán que ciertas categorías se encuentran más cercanas unas de otras, siendo que esto es generado puramente artificialmente por el encoding, y no por las datos per se. 

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

_**Encoding one-hot**_

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [28]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [29]:
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido ``pd.get_dummies``. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

In [32]:
movies_obj = pd.read_csv('vol/intermediate_results/movies_obj.csv')

In [33]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

## Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [34]:
!pip install category_encoders

     |████████████████████████████████| 81kB 326kB/s eta 0:00:01


In [42]:
categoricals = pd.read_csv('vol/intermediate_results/categoricals.csv')

In [43]:
categoricals.head(2)

,Unnamed: 0,actor_1_name,director_name
0,0,CCH Pounder,James Cameron
1,1,Doug Walker,Doug Walker


In [44]:
categoricals = categoricals.reset_index(drop=True).fillna(0)

In [45]:
categoricals.head(2)

,Unnamed: 0,actor_1_name,director_name
0,0,CCH Pounder,James Cameron
1,1,Doug Walker,Doug Walker


In [46]:
X_binenc = pd.concat([X,categoricals],axis=1)

In [68]:
X_binenc.head()
X.drop('Unnamed: 0',axis=1)
X_binenc = X_binenc.drop('Unnamed: 0',axis=1)

In [69]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [70]:
encoder.fit_transform(X_binenc)

,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,actor_1_name_0,actor_1_name_1,actor_1_name_2,...,director_name_2,director_name_3,director_name_4,director_name_5,director_name_6,director_name_7,director_name_8,director_name_9,director_name_10,director_name_11
0,425000000.0,2009.000000,1.780000,178.000000,4834.0,2.370000e+08,7.9,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,306000000.0,2002.130733,2.126976,108.577186,143.0,4.045539e+07,7.1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,300000000.0,2007.000000,2.350000,169.000000,48350.0,3.000000e+08,7.1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,300000000.0,2015.000000,2.350000,148.000000,11700.0,2.450000e+08,6.8,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,275000000.0,2012.000000,2.350000,164.000000,106759.0,2.500000e+08,8.5,0,0,0,...,0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,7000.0,2004.000000,1.850000,77.000000,368.0,7.000000e+03,7.0,0,1,1,...,1,1,1,0,1,0,1,1,0,1
4100,7000.0,2005.000000,2.126976,80.000000,0.0,7.000000e+03,6.3,0,1,1,...,1,1,1,0,1,0,1,1,1,0
4101,7000.0,2005.000000,2.126976,84.000000,93.0,3.250000e+03,7.8,0,1,1,...,1,1,1,0,1,0,1,1,1,1
4102,3967.0,2012.000000,2.350000,100.000000,2386.0,4.045539e+07,6.3,0,0,1,...,1,1,1,0,1,1,0,0,0,0


In [79]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binenc,y)

In [80]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

KeyError: "['Unnamed: 0'] not in index"